In [4]:
import os
import ee
import geojson
import rasterio
import rioxarray
import xarray as xr
import datetime as dt
from funcs_gee import *

# ----------------------------------------------------------------------

# Define the time range.
start_date = '2022-01-01'
end_date = '2022-12-31'
fn=f'NDVI_{start_date}_{end_date}'
geojson_path='/home/martina/Documents/thesis-codes/WATERSTEM/Pascoli_UNIFI_shape/SanLorenzo1.geojson'
geojson_fd, geojson_fn = os.path.split(geojson_path)
geometry_title, ext = os.path.splitext(geojson_fn)

folder_maps='./Maps/'+geometry_title+'/'
if not os.path.exists(folder_maps):
    os.makedirs(folder_maps)


with open(geojson_path) as f:
    gj = geojson.load(f)

nfeatures = len(gj['features'])
coords_geojson = [gj['features'][i]['geometry']['coordinates'] for i in range(nfeatures)]

# Clouds masking
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


# ----------------------------------------------------------------------


def get_s2_sr_cld_col(aoi, start_date, end_date, **kwargs):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))



def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))



def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))



def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)



def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


# ----------------------------------------------------------------------

# Initialize the Earth Engine module.
ee.Initialize()

region = ee.Geometry.MultiPolygon(coords_geojson)

s2=get_s2_sr_cld_col(region, start_date, end_date)

proj=s2.first().select('B2').projection()
scale_mod=ee.Number(proj.nominalScale()).getInfo()
trans_mod=proj.getInfo()['transform']
crs_mod=proj.getInfo()['crs']

# Update masks/add bands over the collection.
s2 = (s2.map(add_cld_shdw_mask)
        .map(apply_cld_shdw_mask))
s2 = s2.map(addNDVI)

# Get pixels in the region with all bands, lon, lat and time
ndvi = np.array(s2.getRegion(region, scale_mod).getInfo())
ndvi[ndvi==None]=np.nan

# Build dataframe and clean timestamps
df = pd.DataFrame(ndvi[1:], columns=ndvi[0])
df['datetime'] = pd.to_datetime([dt.datetime.fromtimestamp(float(ts)/1000).strftime("%Y-%m-%d") for ts in df['time']])
drop_columns=['time','id']
df.drop(drop_columns, axis=1, inplace=True)
df['coords']=list(zip(df.longitude, df.latitude))
dff=df.groupby(by=['datetime','coords']).max()
dff=dff.astype('float')

dates=np.unique(df.datetime)
ds=dff.reset_index().drop(columns=['coords']).set_index(['datetime', 'latitude', 'longitude']).to_xarray()

answ=input('Save nc file? [[y]/n]')
if answ=='y' or answ=='':
    ds.to_netcdf(folder_maps+fn+'.nc')

ds.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)

# Set the Coordinate Reference System (CRS) if it is not set
# You need to know the EPSG code of your CRS
# For example, the EPSG code for WGS84 is 'EPSG:4326'
ds.rio.write_crs("EPSG:4326", inplace=True)

daily_opt=input('Do you want daily images with all bands? (produce first 10) [[y]/n]')
if daily_opt=='' or daily_opt=='y':
    for i, d in enumerate(ds.datetime):
        date_name=str(d.dt.strftime("%Y-%m-%d").values)
        data_daily=ds.sel(datetime=d)
        data_daily.rio.to_raster(folder_maps+date_name+".tif")
        if i==10: break
multi_opt=input('Do you want a single multilayer image with NDVI values only and days as bands? [[y]/n]')
if multi_opt=='' or multi_opt=='y':
    data_var=ds['NDVI']
    data_var.rio.to_raster(folder_maps+fn+".tif")

Save nc file? [[y]/n] n
Do you want daily images with all bands? (produce first 10) [[y]/n] n
Do you want a single multilayer image with NDVI values only and days as bands? [[y]/n] n


In [3]:
from mapsy import *

ModuleNotFoundError: No module named 'cartopy'